In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display


In [ ]:
load_dotenv(override=True)
gpt = OpenAI()
llama = OpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1"
)
gpt_model = "gpt-4o-mini"
llama_model = "llama3.2"

In [ ]:

class Classroom:

    def __init__(self, topic="LLM", display_handle = display(Markdown(""), display_id=True), response = ""):
        self.display_handle = display_handle
        self.response = response

        self.tutor_system = f"You are the tutor who is expert in {topic}. You know best practices in how to impart knowledge on amateur and pro students in very organized way. You first declare the contents of your message separately for amateur and pro student, and then you list down the information in the same order in very organized way such that it's very readable and easy to understand.you highlight the key points every time. you explain with examples, and you have a quite good sense of humor, which you include in your examples and way of tutoring as well. You wait for go ahead from all your students before you move next to the new topic"

        self.amateur_system = f"You are a student who is here to learn {topic}. You ask very basic questions(which comes to mind of a person who has heard the topic for the very first time) but you are intelligent and don't ask stupid questions. you put your question in very organized way. Once you understand a topic you ask tutor to move forward with new topic"

        self.pro_system = f"You are expert of {topic}. You cross-question the tutor to dig deeper into the topic, so that nothing inside the topic is left unknown and unmentioned by the tutor. you post your questions in a very organized manner highlighting the keypoints, such that an amateur can also understand your point or query that you are making. You complement the queries made by amateur and dig deeper into the concept ask by him as well. You also analyze the tutor's response such that it doesn't miss anything and suggest improvements in it as well. Once you understand a topic you ask tutor to move forward with new topic"

        self.tutor_messages = ["Hi, I'm an expert on LLMs!"]
        self.amateur_messages = ["Hi, I'm new to LLMs. I just heard someone using this term in office."]
        self.pro_messages = ["Hey, I'm here to brush up my knowledge on LLMs and gain a more deeper understanding of LLMs"]
        
    def call_tutor(self):
        messages = [
            {"role": "system", "content": self.tutor_system}
        ]
        for tutor, amateur, pro in zip(self.tutor_messages, self.amateur_messages, self.pro_messages):
            messages.append({"role": "assistant", "content": f"tutor: {tutor}"})
            messages.append({"role": "user", "content": f"amateur: {amateur}"})
            messages.append({"role": "user", "content": f"pro: {pro}"})

        if len(self.amateur_messages) > len(self.tutor_messages):
            messages.append({"role": "user", "content": f"amateur: {self.amateur_messages[-1]}"})

        if len(self.pro_messages) > len(self.tutor_messages):
            messages.append({"role": "user", "content": f"amateur: {self.pro_messages[-1]}"})

        stream = llama.chat.completions.create(
            model = llama_model,
            messages = messages,
            stream=True
        )
        self.response += "\n\n\n# Tutor: \n"
        response = ""
        for chunk in stream:
            self.response += chunk.choices[0].delta.content or ''
            response += chunk.choices[0].delta.content or ''
            update_display(Markdown(self.response), display_id=self.display_handle.display_id)
        
        self.tutor_messages.append(response)



    def call_amateur(self):
        messages = [
            {"role": "system", "content": self.amateur_system}
        ]
        for tutor, amateur, pro in zip(self.tutor_messages, self.amateur_messages, self.pro_messages):
            messages.append({"role": "user", "content": f"tutor: {tutor}"})
            messages.append({"role": "assistant", "content": f"amateur: {amateur}"})
            messages.append({"role": "user", "content": f"pro: {pro}"})

        if len(self.tutor_messages) > len(self.amateur_messages):
            messages.append({"role": "user", "content": f"amateur: {self.tutor_messages[-1]}"})

        if len(self.pro_messages) > len(self.amateur_messages):
            messages.append({"role": "user", "content": f"amateur: {self.pro_messages[-1]}"})

        stream = llama.chat.completions.create(
            model = llama_model,
            messages = messages,
            stream=True
        )
        self.response += "\n\n\n# Amateur: \n"
        response = ""
        for chunk in stream:
            self.response += chunk.choices[0].delta.content or ''
            response += chunk.choices[0].delta.content or ''
            update_display(Markdown(self.response), display_id=self.display_handle.display_id)
        
        self.amateur_messages.append(response)



    def call_pro(self):
        messages = [
            {"role": "system", "content": self.pro_system}
        ]
        for tutor, amateur, pro in zip(self.tutor_messages, self.amateur_messages, self.pro_messages):
            messages.append({"role": "user", "content": f"tutor: {tutor}"})
            messages.append({"role": "user", "content": f"amateur: {amateur}"})
            messages.append({"role": "assistant", "content": f"pro: {pro}"})
            
        if len(self.tutor_messages) > len(self.pro_messages):
            messages.append({"role": "user", "content": f"amateur: {self.tutor_messages[-1]}"})

        if len(self.amateur_messages) > len(self.pro_messages):
            messages.append({"role": "user", "content": f"amateur: {self.amateur_messages[-1]}"})

        stream = llama.chat.completions.create(
            model = llama_model,
            messages = messages,
            stream=True
        )
        self.response += "\n\n\n# Pro: \n"
        response = ""
        for chunk in stream:
            response = chunk.choices[0].delta.content or ''
            self.response +=  response
            update_display(Markdown(self.response), display_id=self.display_handle.display_id)

        self.pro_messages.append(response)

    def discuss(self, n=5):
        for i in range(n):
            self.call_tutor()
            self.call_amateur()
            self.call_pro()
cls = Classroom("LLM")
cls.discuss()